In [ ]:
import torch
if torch.cuda.is_available():
    print("GPU is available!")
    print("Device Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available.")

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import deepxde as dde
import os

# Set the backend to PyTorch
os.environ['DDE_BACKEND'] = 'pytorch'
dde.config.set_random_seed(42)

## 1. Problem Definition: Geometry, PDE, and Boundary Conditions

In [ ]:
# Define the Annulus Geometry
# Inner radius r1 = 1, Outer radius r2 = 2
r1 = 1.0
r2 = 2.0

disk_outer = dde.geometry.Disk([0, 0], r2)
disk_inner = dde.geometry.Disk([0, 0], r1)
geom = dde.geometry.CSGDifference(disk_outer, disk_inner)

# Define the PDE (Laplace equation for now: ∇²u = 0)
def pde(x, u):
    """
    Defines the residual of the Laplace equation: ∇²u = 0.
    Args:
        x: A tensor of coordinates (x, y) with shape [N, 2].
        u: The network's output tensor u(x, y) with shape [N, 1].
    Returns:
        The PDE residual tensor.
    """
    dudx2 = dde.grad.hessian(u, x, i=0, j=0)
    dudy2 = dde.grad.hessian(u, x, i=1, j=1)
    laplacian = dudx2 + dudy2
    return laplacian

# Define Boundary Conditions
def boundary_outer(x, on_boundary):
    # x is a [N, 2] array for (x,y) coordinates
    # Check if points are on the outer circle (r=r2)
    return on_boundary and np.isclose(np.sqrt(x[0]**2 + x[1]**2), r2)

def boundary_inner(x, on_boundary):
    # Check if points are on the inner circle (r=r1)
    return on_boundary and np.isclose(np.sqrt(x[0]**2 + x[1]**2), r1)

def func_outer(x):
    # u = 0 on the outer boundary
    return 0

def func_inner(x):
    # u = sin(theta) on the inner boundary
    # x_val = x[:, 0:1]
    # y_val = x[:, 1:2]
    x_val = x[:, 0]
    y_val = x[:, 1]
    theta = np.arctan2(y_val, x_val) # theta = arctan(y/x)
    return np.sin(theta).reshape(-1,1) # Ensure output shape is [N,1]

bc_outer = dde.DirichletBC(geom, func_outer, boundary_outer)
bc_inner = dde.DirichletBC(geom, func_inner, boundary_inner)

bcs = [bc_outer, bc_inner]

## 2. Create the Model and Data

In [ ]:
data = dde.data.PDE(
    geom,
    pde,
    bcs,
    num_domain=2500, # Number of points inside the domain
    num_boundary=800,  # Number of points on the boundary
    num_test=1000,     # Number of points for testing (if solution is known)
    # solution=analytical_solution # Add if an analytical solution is available for error metric
)

# Define the neural network
net = dde.nn.FNN([2] + [50] * 4 + [1], "tanh", "Glorot normal")

# Create the model
model = dde.Model(data, net)

## 3. Train the Model

In [ ]:
# Compile and train the model (Adam)
model.compile("adam", lr=1e-3)
losshistory_adam, train_state_adam = model.train(iterations=15000)

# Compile and train the model (L-BFGS)
model.compile("L-BFGS")
losshistory_lbfgs, train_state_lbfgs = model.train()

## 4. Visualize the Results

In [ ]:
def plot_loss_history(losshistory, title_suffix):
    loss_train = np.sum(losshistory.loss_train, axis=1)
    loss_test = np.sum(losshistory.loss_test, axis=1)
    steps = losshistory.steps

    plt.figure(figsize=(10, 6))
    plt.plot(steps, loss_train, label='Train Loss', color='blue')
    plt.plot(steps, loss_test, label='Test Loss', color='red', linestyle='--')
    plt.yscale('log')
    plt.title(f'Training and Test Loss History ({title_suffix})')
    plt.xlabel('Optimization Step')
    plt.ylabel('Log-scale Loss')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.tight_layout()
    plt.savefig(f"poisson_annulus_loss_history_{title_suffix.lower()}.png")
    plt.show()

plot_loss_history(losshistory_adam, "Adam")
plot_loss_history(losshistory_lbfgs, "L-BFGS")

# Generate points for plotting
n_points = 200
x_range = np.linspace(-r2, r2, n_points)
y_range = np.linspace(-r2, r2, n_points)
grid_x, grid_y = np.meshgrid(x_range, y_range)
plot_points_cartesian = np.hstack((grid_x.flatten()[:, None], grid_y.flatten()[:, None]))

# Keep only points inside the annulus
is_inside_annulus = geom.inside(plot_points_cartesian)
plot_points_annulus = plot_points_cartesian[is_inside_annulus]

# Make predictions
u_pred_annulus = model.predict(plot_points_annulus)

# 2D Contour Plot
plt.figure(figsize=(8, 8))
triang = tri.Triangulation(plot_points_annulus[:, 0], plot_points_annulus[:, 1])
contour = plt.tricontourf(triang, u_pred_annulus.flatten(), levels=100, cmap='viridis')
plt.colorbar(contour, label='u(x,y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('PINN Predicted Solution on Annulus')
plt.axis('equal')
plt.savefig("poisson_annulus_solution_2D.png")
plt.show()

# 3D Surface Plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(plot_points_annulus[:, 0], plot_points_annulus[:, 1], u_pred_annulus.flatten(), cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('u(x,y)')
ax.set_title('3D PINN Predicted Solution on Annulus')
plt.savefig("poisson_annulus_solution_3D.png")
plt.show()